# Emailability

This notebook covers [T214124](https://phabricator.wikimedia.org/T214124), where I should investigate the following:

"Newcomers who have a verified email AND who responded on the welcome survey that they are interested in receiving contact from mentor.

We need to see this separately for Korean and Czech, amongst just those newcomers who were in the treatment group for the welcome survey."

I'll reuse much of the initial code from the Welcome Survey, but extend the data gathering until the most recent full week.

In [1]:
import json

import datetime as dt
import pandas as pd

from wmfdata import mariadb, hive

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [2]:
%load_ext rpy2.ipython

In [43]:
## Configuration variables

## Czech Wikipedia setup:

## User IDs of known users to exclude (Stephane, Elena, and Marshall's accounts)
cs_known_users = set([322106, 339583, 341191, 341611, 433381, 433382, 433511, 404765, 421667,
                      427625, 437386])

## The start timestamp is the registration timestamp of Roan's test account (which we'll
## filter out later).
cs_start_timestamp = dt.datetime(2018, 11, 19, 20, 0, 0)

## We set the end timestamp to be midnight Dec 25. This is mainly motivated by a spambot attack on
## Korean Wikipedia, which appears to have started some time on Dec 28. Using Dec 25 means we
## measure a set of whole weeks, which matches the weekly cycle that Wikipedias tend to move in.
cs_end_timestamp = dt.datetime(2018, 12, 25, 0, 0, 0)

## Korean Wikipedia setup:

## User IDs of known users to exclude
ko_known_users = set([384066, 539296, 539299, 539302, 539303, 539304, 539305, 539306, 539307,
                      539298, 416361, 416360, 413162, 495265, 518393, 518394, 518396, 530285,
                      531579, 531785, 536786, 536787, 542720, 542721, 542722, 543192, 543193,
                      544145, 544283])

## Start timestamp is some time prior to the registration timestamp of the first user who
## got the survey. In this case, Roan's timestamp works just fine.
ko_start_timestamp = dt.datetime(2018, 11, 19, 20, 0, 0)
ko_end_timestamp = cs_end_timestamp

## Format strings:
## MediaWiki database timestamp format
mw_format = "%Y%m%d%H%M%S"
hive_format = "%Y-%m-%d %H:%M%:S"


In [44]:
## Grab the user IDs of all accounts of Growth Team members (particularly their test accounts)
## so that they can be excluded from the analysis.

username_patterns = ["MMiller", "Zilant", "Roan", "KHarlan", "MWang"]

known_user_query = '''
SELECT user_id
FROM {wiki}.user
WHERE user_registration >= "{start_timestamp}"
AND user_name LIKE "{name_pattern}%"
'''

for u_pattern in username_patterns:
    cs_known = mariadb.run(known_user_query.format(
        wiki = 'cswiki',
        start_timestamp = cs_start_timestamp.strftime(mw_format),
        name_pattern = u_pattern), fmt = 'raw')
    cs_known_users = cs_known_users | set([r[0] for r in cs_known])

for u_pattern in username_patterns:
    ko_known = mariadb.run(known_user_query.format(
        wiki = 'kowiki',
        start_timestamp = cs_start_timestamp.strftime(mw_format),
        name_pattern = u_pattern), fmt = 'raw')
    ko_known_users = ko_known_users | set([r[0] for r in ko_known])

In [9]:
## Query to retrieve the user ID and whether the registration was on the mobile site
## for all non-autocreated non-app registrations between the given timestamps for the given wiki.

mob_query = '''SELECT event.userid, event.displaymobile
FROM event.serversideaccountcreation
WHERE year = 2018
AND month >= 11
AND dt BETWEEN "{start_timestamp}" AND "{end_timestamp}"
AND event.isapi = 0
AND event.isselfmade = 1
AND wiki = "{wiki}"
AND event.userid NOT IN ({idlist})
'''

cs_users_raw = hive.run(mob_query.format(
  start_timestamp = cs_start_timestamp.strftime(hive_format),
  end_timestamp = cs_end_timestamp.strftime(hive_format),
  wiki = 'cswiki',
  idlist = ','.join([str(uid) for uid in cs_known_users])))

ko_users_raw = hive.run(mob_query.format(
  start_timestamp = ko_start_timestamp.strftime(hive_format),
  end_timestamp = ko_end_timestamp.strftime(hive_format),
  wiki = 'kowiki',
  idlist = ','.join([str(uid) for uid in ko_known_users])))

In [11]:
## Query to MW database to get info on all users who were in the survey group

group_query = '''
SELECT up_user,
  CASE
    WHEN up_value LIKE "%exp1_group2%" THEN "control"
    WHEN up_value LIKE "%exp1_group1%" THEN "target"
  END AS exp_group
FROM {wiki}.user_properties
WHERE up_property = "welcomesurvey-responses"
AND up_user IN ({id_list})
AND up_value <> ""
'''

cs_groups_raw = mariadb.run(group_query.format(
    wiki = 'cswiki',
    start_timestamp = cs_start_timestamp,
    end_timestamp = cs_end_timestamp,
    id_list = ",".join([str(id) for id in cs_users_raw['userid']])))

ko_groups_raw = mariadb.run(group_query.format(
    wiki = 'kowiki',
    start_timestamp = ko_start_timestamp,
    end_timestamp = ko_end_timestamp,
    id_list = ",".join([str(id) for id in ko_users_raw['userid']])))

In [18]:
## 1: responded that they were open to be contacted by a mentor

mentor_query = '''
SELECT up_user
FROM {wiki}.user_properties
WHERE up_property = 'welcomesurvey-responses' 
AND up_value LIKE '%"mentor":true%exp1_group1%'
AND up_user IN ({id_list})
'''

cs_mentor = mariadb.run(mentor_query.format(
    wiki = 'cswiki',
    id_list = ",".join(
        [str(id) for id in
         cs_groups_raw.loc[cs_groups_raw['exp_group'] == 'target']['up_user']])))

ko_mentor = mariadb.run(mentor_query.format(
    wiki = 'kowiki',
    id_list = ",".join(
        [str(id) for id in
         ko_groups_raw.loc[ko_groups_raw['exp_group'] == 'target']['up_user']])))

In [46]:
## 2: have a verified email address

verified_email_query = '''
SELECT user_id
FROM {wiki}.user
WHERE user_id IN ({id_list})
AND user_email_authenticated IS NOT NULL
'''

cs_verified = mariadb.run(verified_email_query.format(
    wiki = 'cswiki',
    id_list = ",".join(
        [str(id) for id in
         cs_groups_raw.loc[cs_groups_raw['exp_group'] == 'target']['up_user']])))

ko_verified = mariadb.run(verified_email_query.format(
    wiki = 'kowiki',
    id_list = ",".join(
        [str(id) for id in
         ko_groups_raw.loc[ko_groups_raw['exp_group'] == 'target']['up_user']])))


In [47]:
## Add relevant columns to the opt-in and verified email datasets

cs_mentor['mentor_opt_in'] = True
ko_mentor['mentor_opt_in'] = True

cs_verified['has_verified_email'] = True
ko_verified['has_verified_email'] = True

In [48]:
## Left join all the things

cs_users = pd.merge(cs_groups_raw.loc[cs_groups_raw['exp_group'] == 'target'],
                    cs_mentor, how = 'left', on = 'up_user')
cs_users = pd.merge(cs_users, cs_verified, how = 'left', left_on = 'up_user', right_on = 'user_id')

ko_users = pd.merge(ko_groups_raw.loc[ko_groups_raw['exp_group'] == 'target'],
                    ko_mentor, how = 'left', on = 'up_user')
ko_users = pd.merge(ko_users, ko_verified, how = 'left', left_on = 'up_user', right_on = 'user_id')

cs_users = cs_users.drop(['exp_group', 'user_id'], axis = 1)
ko_users = ko_users.drop(['exp_group', 'user_id'], axis = 1)

cs_users = cs_users.fillna(False)
ko_users = ko_users.fillna(False)

In [51]:
## Number of users in the survey group:

cs_users['up_user'].nunique()

816

In [53]:
## Number of users who meet both conditions:
cs_users.loc[(cs_users['mentor_opt_in'] == True) &
                   (cs_users['has_verified_email'] == True)]['up_user'].nunique()


51

In [49]:
## So, what proportion of all survey users have those two properties set?
## Czech Wikipedia:
round(100 *
      cs_users.loc[(cs_users['mentor_opt_in'] == True) &
                   (cs_users['has_verified_email'] == True)]['up_user'].nunique() /
      cs_users['up_user'].nunique(), 1)

6.2

In [56]:
## How many Czech users have a verified email address?

cs_users.loc[cs_users['has_verified_email'] == True]['up_user'].nunique()

202

In [58]:
## How many Czech users opted in?

cs_users.loc[cs_users['mentor_opt_in'] == True]['up_user'].nunique()

188

In [52]:
## Same, but for Korean:

ko_users['up_user'].nunique()

1063

In [54]:
ko_users.loc[(ko_users['mentor_opt_in'] == True) &
                   (ko_users['has_verified_email'] == True)]['up_user'].nunique()

76

In [50]:
## Korean Wikipedia:
round(100 *
      ko_users.loc[(ko_users['mentor_opt_in'] == True) &
                   (ko_users['has_verified_email'] == True)]['up_user'].nunique() /
      ko_users['up_user'].nunique(), 1)

7.1

In [57]:
## How many Korean users have a verified email address?

ko_users.loc[ko_users['has_verified_email'] == True]['up_user'].nunique()

150

In [59]:
## How many Korean users opted in?

ko_users.loc[ko_users['mentor_opt_in'] == True]['up_user'].nunique()

329